In [2]:
from pycaret.time_series import load_model
model = load_model("ts_model", "azure", {"container": "models"})

Transformation Pipeline and Model Successfully Loaded


In [5]:
from azure.storage.blob import BlobClient
import pandas as pd
import os
blob = BlobClient.from_connection_string(conn_str=os.environ["AZURE_STORAGE_CONNECTION_STRING"], 
                                            container_name="data", 
                                            blob_name="processed_data.csv")
blob_data = blob.download_blob()
data = pd.read_csv(blob_data, parse_dates=['date'], index_col="date")
data.rename({"total": "actual"}, axis=1, inplace=True)

In [8]:
import datetime
cutoff = pd.Timestamp("2023-04-28")
start_date = datetime.datetime(2023, 5, 8)
end_date = datetime.datetime(2023, 5, 12)
pred_data = data.loc[start_date:end_date]

In [9]:
pred_data

,acc_precip,isHoliday,isSchoolHoliday,mean_cloud_cover,mean_relative_hum,mean_temp,mean_wind_speed,out-of-office,actual,dow_0,dow_1,dow_2,dow_3,dow_4,total_lag7
date,,,,,,,,,,,,,,,
2023-05-08,0.0,0,0,100.0,79.6,11.0,4.0,3,NaN,1,0,0,0,0,174.0
2023-05-09,0.0,0,0,34.4,77.3,11.3,3.7,1,NaN,0,1,0,0,0,227.0
2023-05-10,0.5,0,0,100.0,93.3,11.8,2.8,2,NaN,0,0,1,0,0,225.0
2023-05-11,0.0,0,0,100.0,89.7,11.2,2.4,5,NaN,0,0,0,1,0,NaN
2023-05-12,0.0,0,0,100.0,87.9,11.5,2.8,9,NaN,0,0,0,0,1,NaN


In [11]:
import numpy as np
model.predict(fh=np.arange(1,5), X=pred_data)

ValueError: BaseCdsDtForecaster cannot handle missing data (nans), but X passed contained missing data.